In [30]:
import pandas as pd
# Reset the options to default
# Set the maximum number of columns to display
pd.set_option('display.max_columns', 500)

# Set the maximum number of rows to display
pd.set_option('display.max_rows', None)

In [16]:
hprc_biosample = pd.read_csv('/private/groups/hprc/human-pangenomics/documentation-metadata/HPRC_metadata/data/production/hprc-production-biosample-table.tsv', sep='\t')
hprc_biosample.rename(columns={'Sample':'sample_ID'}, inplace=True)

In [2]:
wustl_y1_topup_sra = pd.read_csv('HPRC_WUSTL_Y1_PacBio_HiFi_TopUp_Metadata_Submission_v0.1.tsv',sep='\t')

In [3]:
wustl_y1_topup_sra['sample_id'] = wustl_y1_topup_sra['sample_ID'] + '-' + wustl_y1_topup_sra['filename']

In [5]:
# List of column names from your metadata sheet
metadata_columns = [
    'filename', 'library_ID', 'library_strategy', 'library_source', 
    'library_selection', 'library_layout', 'platform', 'instrument_model', 
    'design_description', 'data_type', 'shear_method', 'size_selection', 
    'ccs_algorithm', 'polymerase_version', 'seq_plate_chemistry_version', 
    'generator_facility', 'generator_contact', 'biosample_accession', 
    'title', 'filetype', 'assembly'
]

# Assume you have another DataFrame named 'other_df'
# Example: other_df = pd.read_csv('path_to_other_file.csv')

# Check if the metadata columns are present in the other DataFrame
missing_columns = [col for col in metadata_columns if col not in wustl_y1_topup_sra.columns]

# Output the missing columns
print("Missing columns:", missing_columns)


Missing columns: ['biosample_accession', 'title', 'assembly']


In [6]:
wustl_y1_topup_output_df = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/submissions/WUSTL_HPRC_HiFi_Year1_TopUp/hifi_qc/hifi_qc_samples_output_table.csv')

In [10]:
wustl_y1_topup_sra = wustl_y1_topup_sra[wustl_y1_topup_sra['sample_id'].isin(wustl_y1_topup_sra['sample_id'].tolist())]

In [14]:
assert wustl_y1_topup_output_df.shape[0] == wustl_y1_topup_sra.shape[0]

In [15]:
wustl_y1_topup_sra['assembly'] = 'unaligned'

In [17]:
wustl_y1_topup_sra = pd.merge(hprc_biosample, wustl_y1_topup_sra, on='sample_ID', how='inner')

In [18]:
wustl_y1_topup_sra['title'] = 'PacBio HiFi sequencing of ' + wustl_y1_topup_sra['sample_ID']

In [21]:
wustl_y1_topup_sra.rename(columns={'Accession':'biosample_accession'},inplace=True)

In [22]:
wustl_y1_topup_sra = wustl_y1_topup_sra[metadata_columns]

In [23]:
wustl_y1_topup_sra['instrument_model'] = 'Revio'

/data/tmp/ipykernel_1830256/113464685.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wustl_y1_topup_sra['instrument_model'] = 'Revio'


In [24]:
wustl_y1_topup_sra['library_ID'] = wustl_y1_topup_sra['library_ID'] + '_' + wustl_y1_topup_sra['filename']

/data/tmp/ipykernel_1830256/1043829791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wustl_y1_topup_sra['library_ID'] = wustl_y1_topup_sra['library_ID'] + '_' + wustl_y1_topup_sra['filename']


In [25]:
wustl_y1_topup_sra['library_layout'] = 'single'

/data/tmp/ipykernel_1830256/2457177396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wustl_y1_topup_sra['library_layout'] = 'single'


In [31]:
wustl_y1_topup_sra.to_csv('sra_biosample_WUSTL_Y1_TopUp.tsv',index=False, sep='\t')

,filename,library_ID,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,data_type,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact,biosample_accession,title,filetype,assembly
0,m84081_231222_081401_s1.hifi_reads.bc2028.bam,HG00673_lib1_m84081_231222_081401_s1.hifi_read...,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,Revio,HiFi sequencing of 20kb fractionated gDNA,unaligned reads,Megaruptor 1,SageELF,8.0.0,Revio P1,Revio C1,Washington University,tgraves@wustl.edu,SAMN17861654,PacBio HiFi sequencing of HG00673,bam,unaligned
1,m84081_231222_101332_s2.hifi_reads.bc2028.bam,HG00673_lib1_m84081_231222_101332_s2.hifi_read...,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,Revio,HiFi sequencing of 20kb fractionated gDNA,unaligned reads,Megaruptor 1,SageELF,8.0.0,Revio P1,Revio C1,Washington University,tgraves@wustl.edu,SAMN17861654,PacBio HiFi sequencing of HG00673,bam,unaligned
2,m84081_231222_121301_s3.hifi_reads.bc2028.bam,HG00673_lib1_m84081_231222_121301_s3.hifi_read...,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,Revio,HiFi sequencing of 20kb fractionated gDNA,unaligned reads,Megaruptor 1,SageELF,8.0.0,Revio P1,Revio C1,Washington University,tgraves@wustl.edu,SAMN17861654,PacBio HiFi sequencing of HG00673,bam,unaligned
3,m84081_231222_141231_s4.hifi_reads.bc2028.bam,HG00673_lib1_m84081_231222_141231_s4.hifi_read...,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,Revio,HiFi sequencing of 20kb fractionated gDNA,unaligned reads,Megaruptor 1,SageELF,8.0.0,Revio P1,Revio C1,Washington University,tgraves@wustl.edu,SAMN17861654,PacBio HiFi sequencing of HG00673,bam,unaligned
4,m84081_231222_081401_s1.hifi_reads.bc2026.bam,HG01071_lib1_m84081_231222_081401_s1.hifi_read...,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,Revio,HiFi sequencing of 20kb fractionated gDNA,unaligned reads,Megaruptor 1,SageELF,8.0.0,Revio P1,Revio C1,Washington University,tgraves@wustl.edu,SAMN17861657,PacBio HiFi sequencing of HG01071,bam,unaligned
5,m84081_231222_101332_s2.hifi_reads.bc2026.bam,HG01071_lib1_m84081_231222_101332_s2.hifi_read...,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,Revio,HiFi sequencing of 20kb fractionated gDNA,unaligned reads,Megaruptor 1,SageELF,8.0.0,Revio P1,Revio C1,Washington University,tgraves@wustl.edu,SAMN17861657,PacBio HiFi sequencing of HG01071,bam,unaligned
6,m84081_231222_121301_s3.hifi_reads.bc2026.bam,HG01071_lib1_m84081_231222_121301_s3.hifi_read...,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,Revio,HiFi sequencing of 20kb fractionated gDNA,unaligned reads,Megaruptor 1,SageELF,8.0.0,Revio P1,Revio C1,Washington University,tgraves@wustl.edu,SAMN17861657,PacBio HiFi sequencing of HG01071,bam,unaligned
7,m84081_231222_141231_s4.hifi_reads.bc2026.bam,HG01071_lib1_m84081_231222_141231_s4.hifi_read...,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,Revio,HiFi sequencing of 20kb fractionated gDNA,unaligned reads,Megaruptor 1,SageELF,8.0.0,Revio P1,Revio C1,Washington University,tgraves@wustl.edu,SAMN17861657,PacBio HiFi sequencing of HG01071,bam,unaligned
8,m84081_231222_081401_s1.hifi_reads.bc2024.bam,HG00741_lib1_m84081_231222_081401_s1.hifi_read...,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,Revio,HiFi sequencing of 20kb fractionated gDNA,unaligned reads,Megaruptor 1,SageELF,8.0.0,Revio P1,Revio C1,Washington University,tgraves@wustl.edu,SAMN17861656,PacBio HiFi sequencing of HG00741,bam,unaligned
9,m84081_231222_101332_s2.hifi_reads.bc2024.bam,HG00741_lib1_m84081_231222_101332_s2.hifi_read...,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,Revio,HiFi sequencing of 20kb fractionated gDNA,unaligned reads,Megaruptor 1,SageELF,8.0.0,Revio P1,Revio C1,Washington University,tgraves@wustl.edu,SAMN17861656,PacBio HiFi sequencing of HG00741,bam,unaligned
